In [31]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df1 = pd.read_csv('train.csv', low_memory= False, index_col= 0)
df2 = pd.read_csv('test.csv', low_memory= False, index_col= 0)

# Data Checking
## Column Type
print('df1_type_check')
print(df1.info())
print('\ndf2_type_check')
print(df2.info())
## Blank/null Check
print('\ndf1_blank_check')
print((df1.isna().mean()*100)[(df1.isna().mean()*100)>0].sort_values(ascending= False))
print('\ndf2_blank_check')
print((df2.isna().mean()*100)[(df2.isna().mean()*100)>0].sort_values(ascending= False))
# Zero value Check: LotFrontage, LotArea Column
num_cols_need_check = ['LotFrontage', 'LotArea']
def zero_value_check(df):
  report = {}
  for col in num_cols_need_check:
    zero_pct = (df[col] == 0).mean()*100
    if zero_pct > 0:
      report[col] = round(zero_pct, 2)
  return pd.Series(report, name= 'zero_pct').sort_values(ascending= False)
print('\ndf1_zero_value_check')
print(zero_value_check(df1))
print('\ndf2_zero_value_check')
print(zero_value_check(df2))
# Outliers check
def outlier_check(df):
  report= {}
  for col in df.select_dtypes(include= 'number').columns:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).mean()*100
    if outliers > 0:
      report[col] = round(outliers, 2)
  return pd.Series(report, name= 'outlier_pct').sort_values(ascending= False)
print('\ndf1_outlier_check')
print(outlier_check(df1))
print('\ndf2_outlier_check')
print(outlier_check(df2))

# Data Cleaning
## Handling Blank
df1['LotFrontage'] = df1.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
df2['LotFrontage'] = df2.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
df2['MSZoning'] = df2['MSZoning'].fillna(df2['MSZoning'].mode()[0])
df2['SaleType'] = df2['SaleType'].fillna(df2['SaleType'].mode()[0])
print("Remaining missing values in train:", df1.isna().mean()*100)
print("Remaining missing values in test:", df2.isna().mean()*100)

# Build Model
## Define x, y
x = df1.drop(['Alley'], axis= 1)
cat_cols_non_rank_x = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'GarageFinish', 'PavedDrive', 'MiscFeature', 'SaleCondition', 'SaleType']
cat_cols_rank_x = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence']
df1 = pd.get_dummies(df1, columns= cat_cols_non_rank_x, drop_first= True)
qual_map = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, None:0}
exp_map = {"Gd":4, "Av":3, "Mn":2, "No":1, None:0}
fin_map = {"GLQ":6, "ALQ":5, "BLQ":4, "Rec":3, "LwQ":2, "Unf":1, None:0}
func_map = {"Typ":5, "Min1":4, "Min2":3, "Mod":2, "Maj1":1, "Maj2":0, "Sev":-1, "Sal":-2, None:0}
fence_map = {"GdPrv":4,"MnPrv":3,"GdWo":2,"MnWw":1,None:0}

def group_rare(df, col, threshold= 0.05):
  fregs = df[col].value_counts(normalize= True)
  rare_cats = fregs[fregs < threshold].index
  df[col] = df[col].replace(rare_cats, 'Others')
  return df
for col in ['Neighborhood', 'Exterior1st', 'Exterior2nd']:
  df1 = group_rare(df1, col, threshold= 0.05)
  df2 = group_rare(df2, col, threshold= 0.05)
df1 = pd.get_dummies(df1, columns= ['Neighborhood', 'Exterior1st', 'Exterior2nd'], drop_first= True)
df2 = pd.get_dummies(df2, columns= ['Neighborhood', 'Exterior1st', 'Exterior2nd'], drop_first= True)

df1['ExterQual'] = df1['ExterQual'].map(qual_map).fillna(0)
df1['ExterCond'] = df1['ExterCond'].map(qual_map).fillna(0)
df1['BsmtQual'] = df1['BsmtQual'].map(qual_map).fillna(0)
df1['BsmtCond'] = df1['BsmtCond'].map(qual_map).fillna(0)
df1['BsmtExposure'] = df1['BsmtExposure'].map(exp_map).fillna(0)
df1['BsmtFinType1'] = df1['BsmtFinType1'].map(fin_map).fillna(0)
df1['BsmtFinType2'] = df1['BsmtFinType2'].map(fin_map).fillna(0)
df1['HeatingQC'] = df1['HeatingQC'].map(qual_map).fillna(0)
df1['KitchenQual'] = df1['KitchenQual'].map(qual_map).fillna(0)
df1['Functional'] = df1['Functional'].map(func_map).fillna(0)
df1['FireplaceQu'] = df1['FireplaceQu'].map(qual_map).fillna(0)
df1['GarageQual'] = df1['GarageQual'].map(qual_map).fillna(0)
df1['GarageCond'] = df1['GarageCond'].map(qual_map).fillna(0)
df1['PoolQC'] = df1['PoolQC'].map(qual_map).fillna(0)
df1['Fence'] = df1['Fence'].map(fence_map).fillna(0)
df2['ExterQual'] = df2['ExterQual'].map(qual_map).fillna(0)
df2['ExterCond'] = df2['ExterCond'].map(qual_map).fillna(0)
df2['BsmtQual'] = df2['BsmtQual'].map(qual_map).fillna(0)
df2['BsmtCond'] = df2['BsmtCond'].map(qual_map).fillna(0)
df2['BsmtExposure'] = df2['BsmtExposure'].map(exp_map).fillna(0)
df2['BsmtFinType1'] = df2['BsmtFinType1'].map(fin_map).fillna(0)
df2['BsmtFinType2'] = df2['BsmtFinType2'].map(fin_map).fillna(0)
df2['HeatingQC'] = df2['HeatingQC'].map(qual_map).fillna(0)
df2['KitchenQual'] = df2['KitchenQual'].map(qual_map).fillna(0)
df2['Functional'] = df2['Functional'].map(func_map).fillna(0)
df2['FireplaceQu'] = df2['FireplaceQu'].map(qual_map).fillna(0)
df2['GarageQual'] = df2['GarageQual'].map(qual_map).fillna(0)
df2['GarageCond'] = df2['GarageCond'].map(qual_map).fillna(0)
df2['PoolQC'] = df2['PoolQC'].map(qual_map).fillna(0)
df2['Fence'] = df2['Fence'].map(fence_map).fillna(0)
df1, df2 = df1.align(df2, join= 'left', axis= 1, fill_value= 0)

df1 = df1.select_dtypes(include='number')
df2 = df2.select_dtypes(include='number')

x = df1.drop(['SalePrice'], axis= 1)
y = df1['SalePrice']

corr = df1.corr()['SalePrice'].sort_values(ascending= False)
print(corr.head(20))

corr = df1.corr()['SalePrice'].sort_values(ascending= True)
print(corr.head(20))

x = df1[['OverallQual','GrLivArea','ExterQual','KitchenQual','GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','BsmtQual','FullBath','TotRmsAbvGrd','YearBuilt','FireplaceQu','YearRemodAdd','GarageYrBlt','MasVnrArea','Fireplaces', 'HeatingQC','BsmtFinSF1','BsmtCond','BedroomAbvGr','PoolQC','ScreenPorch','Functional','3SsnPorch','MoSold','ExterCond','BsmtFinType2','BsmtFinSF2','BsmtHalfBath','MiscVal','LowQualFinSF','YrSold','OverallCond','MSSubClass','EnclosedPorch','KitchenAbvGr','Fence','PoolArea']]
y = df1['SalePrice']

## Standard Scaler for x
from sklearn.preprocessing import StandardScaler
df1['HouseAge'] = df1['YrSold'] - df1['YearBuilt']
df1['RemodAge'] = df1['YrSold'] - df1['YearRemodAdd']
df1['GarageAge'] = df1['YrSold'] - df1['GarageYrBlt']
df1 = df1.drop(['YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis= 1)

df2['HouseAge'] = df2['YrSold'] - df2['YearBuilt']
df2['RemodAge'] = df2['YrSold'] - df2['YearRemodAdd']
df2['GarageAge'] = df2['YrSold'] - df2['GarageYrBlt']
df2 = df2.drop(['YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis= 1)

scale_columns = ['OverallQual','GrLivArea','GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','MasVnrArea','Fireplaces','BsmtFinSF1','BedroomAbvGr','ScreenPorch','3SsnPorch','MoSold','BsmtFinSF2','BsmtHalfBath','MiscVal','LowQualFinSF','OverallCond','MSSubClass','EnclosedPorch','KitchenAbvGr','PoolArea', 'HouseAge', 'RemodAge', 'GarageAge']
scaler = StandardScaler()
x_scale = scaler.fit_transform(df1[scale_columns])
other_columns = [col for col in x if col not in scale_columns]
x_scale_num = pd.DataFrame(x_scale, columns= scale_columns, index= x.index)
x_combine = pd.concat([x_scale_num, x[other_columns]], axis= 1, join= 'inner', ignore_index= False)
x_combine['GarageAge'] = x_combine['GarageAge'].fillna(0)
x_combine['MasVnrArea'] = x_combine['MasVnrArea'].fillna(0)
x_final = x_combine.drop(['YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis= 1, errors='ignore')


x_test_file = df2[['OverallQual','GrLivArea','ExterQual','KitchenQual','GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','BsmtQual','FullBath','TotRmsAbvGrd','FireplaceQu','MasVnrArea','Fireplaces', 'HeatingQC','BsmtFinSF1','BsmtCond','BedroomAbvGr','PoolQC','ScreenPorch','Functional','3SsnPorch','MoSold','ExterCond','BsmtFinType2','BsmtFinSF2','BsmtHalfBath','MiscVal','LowQualFinSF','OverallCond','MSSubClass','EnclosedPorch','KitchenAbvGr','Fence','PoolArea']]
scale_columns_test_file = ['OverallQual','GrLivArea','GarageCars','GarageArea', 'TotalBsmtSF','1stFlrSF','FullBath','TotRmsAbvGrd','MasVnrArea','Fireplaces','BsmtFinSF1','BedroomAbvGr','ScreenPorch','3SsnPorch','MoSold','BsmtFinSF2','BsmtHalfBath','MiscVal','LowQualFinSF','OverallCond','MSSubClass','EnclosedPorch','KitchenAbvGr','PoolArea', 'HouseAge', 'RemodAge', 'GarageAge']
x_scale_test_file = scaler.transform(df2[scale_columns_test_file])
other_columns_test_file = [col for col in x_test_file if col not in scale_columns_test_file]
x_scale_num_test_file = pd.DataFrame(x_scale_test_file, columns= scale_columns_test_file, index= df2.index)
x_combine_test_file = pd.concat([x_scale_num_test_file, x_test_file[other_columns_test_file]], axis= 1, join= 'inner', ignore_index= False)
x_combine_test_file['GarageAge'] = x_combine_test_file['GarageAge'].fillna(0)
x_combine_test_file['MasVnrArea'] = x_combine_test_file['MasVnrArea'].fillna(0)
x_final_test_file = x_combine_test_file.drop(['YrSold', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], axis= 1, errors='ignore')
x_final_test_file = x_combine_test_file.reindex(columns=x_final.columns, fill_value=0)
print("Train shape:", x_final.shape)
print("Test shape:", x_final_test_file.shape)

## Train Test split
from sklearn.model_selection import train_test_split
x_final_train, x_final_test, y_train, y_test = train_test_split(x_final, y, test_size= 0.2, random_state= 42)

## Linear Regression
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression().fit(x_final_train, y_train)
y_pred_lr = lr_model.predict(x_final_test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
print('Linear Regression Result')
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_score_lr = r2_score(y_test, y_pred_lr)
mape_lr = mean_absolute_percentage_error(y_test, y_pred_lr)
print('MAE:', mae_lr)
print('RMSE:', rmse_lr)
print('R2 Score:', r2_score_lr)
print('MAPE:', mape_lr)

coefficients = lr_model.coef_
intercept = lr_model.intercept_
feature_name = x_final_train.columns
coef_df = pd.DataFrame({'Feature': feature_name, 'Coefficient': coefficients}).sort_values(by = 'Coefficient', ascending= False)
print('Intercept:', intercept)
print(coef_df)

## Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators= 200, max_depth= None, min_samples_split= 2, min_samples_leaf= 3, max_features= 'sqrt', n_jobs= -1).fit(x_final_train, y_train)
y_pred_rf = rf_model.predict(x_final_test)
print('Random Forest Regressor Result')
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_score_rf = r2_score(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
print('MAE:', mae_rf)
print('RMSE:', rmse_rf)
print('R2 Score:', r2_score_rf)
print('MAPE:', mape_rf)

fea_importance = pd.DataFrame({'Feature': x_final.columns, 'Importances': rf_model.feature_importances_}).sort_values(by = 'Importances', ascending= False)
print(fea_importance)

## XGBoost Regressor
from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators= 200, learning_rate= 0.1, max_depth= 5, subsample= 0.8, colsample_bytree= 0.8, random_state= 42, eval_metric= 'rmse', n_jobs= -1, tree_method= 'hist').fit(x_final_train, y_train)
y_pred_xgb = xgb_model.predict(x_final_test)
print('XGBoost Regressor Result')
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_score_xgb = r2_score(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)
print('MAE:', mae_xgb)
print('RMSE:', rmse_xgb)
print('R2 Score:', r2_score_xgb)
print('MAPE:', mape_xgb)

# Predict SalePrice
df2['SalePricePredict'] = xgb_model.predict(x_final_test_file)
df2.to_csv('predictive_test_file.csv', index= False)
files.download('predictive_test_file.csv')

Saving test.csv to test (26).csv
Saving train.csv to train (25).csv
df1_type_check
<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    146

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>